In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import tensorflow as tf
from tensorflow.keras import backend as keras_backend
import numpy as np

gpu_on = True

if gpu_on :
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

print(gpu_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from src.data_loader.classification import ClassifyDataloader
from tensorflow.keras.applications.inception_v3 import preprocess_input
from glob import glob

task = "classification"
data_set_name = "detect_lvi"
batch_size = 32
on_memory = True
argumentation_proba = 0.75
target_size = (512,512)
interpolation = "bilinear"
class_mode = "binary"
# class_mode = "categorical"
dtype="float32"


train_image_path_list = glob(f"./datasets/{task}/{data_set_name}/train/*/*")
valid_image_path_list = glob(f"./datasets/{task}/{data_set_name}/valid/*/*")
test_image_path_list = glob(f"./datasets/{task}/{data_set_name}/test/*/*")
label_list = os.listdir(f"./datasets/{task}/{data_set_name}/train")

label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}

train_data_loader = ClassifyDataloader(image_path_list=train_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=on_memory,
                                       argumentation_proba=argumentation_proba,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,
                                       shuffle=True,
                                       class_mode=class_mode,
                                       dtype=dtype
)
valid_data_loader = ClassifyDataloader(image_path_list=valid_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=False,
                                       argumentation_proba=0,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)
test_data_loader = ClassifyDataloader(image_path_list=test_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=False,
                                       argumentation_proba=0,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)

Total data num 8608 with 2 classes
Total data num 1609 with 2 classes
Total data num 1469 with 2 classes
train_num: 8608
valid_num: 1609
test_num: 1469


In [3]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

import numpy as np
np.random.seed(1337)  # for reproducibility

DROPOUT_RATIO = 0.5

grad_cam = False
transfer_learning = False
epoch_release_frozen = 10
transfer_train_mode = "include_deep_layer"
layer_name_frozen_to = "mixed4"

#  binary_sigmoid, categorical_sigmoid, categorical_softmax
activation = "binary_sigmoid"

# create the base pre-trained model~
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(None,None,3),
    classes=None,
    pooling=None,
    classifier_activation=None
)

if transfer_learning:
    if train_mode == "dense_only":
        base_model.trainable = False
    elif train_mode == "include_deep_layer":
        for layer in base_model.layers: 
            layer.trainable = False
            if layer.name == layer_name_frozen_to:
                break

# add a global spatial average pooling layer
x = base_model.output
# (Batch_Size,?)
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT_RATIO)(x)
# let's add a fully-connected layer
# (Batch_Size,1)
x = Dense(1024, activation='relu')(x)
# (Batch_Size,1024)
x = Dropout(DROPOUT_RATIO)(x)

if grad_cam:
    x *= 1e-1
    keras_backend.set_floatx('float64')
    dense_dtype = "float64"
else:
    dense_dtype = "float32"
    
if activation == "binary_sigmoid":
    predictions = Dense(1, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = BinaryCrossentropy(label_smoothing=0.01)
elif activation == "categorical_sigmoid":
    predictions = Dense(2, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)
elif activation == "categorical_softmax":
    predictions = Dense(2, activation='softmax', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)

# this is the model we will train
model = Model(base_model.input, predictions)

In [4]:
from datetime import date

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import Nadam

today = date.today()

# YY/MM/dd
today_str = today.strftime("%Y-%m-%d")
today_weight_path = f"./weights/{task}/{data_set_name}/{today_str}/" 
today_logs_path = f"./logs/{task}/{data_set_name}/{today_str}/"
os.makedirs(today_weight_path, exist_ok=True)
os.makedirs(today_logs_path, exist_ok=True)
optimizer = Nadam(1e-3, clipnorm=1)

save_c = ModelCheckpoint(
    today_weight_path+"/weights_{epoch:02d}_{loss:.4f}.hdf5",
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


reduceLROnPlat = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=15,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-7)
csv_logger = CSVLogger(f'{today_logs_path}/log.csv', append=False, separator=',')


def make_model_trainable(epoch, model, target_epoch):
    if epoch > target_epoch:
        for layer in model.layers:
            layer.trainable = True
            
make_trainable_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: make_model_trainable(epoch, model, target_epoch=epoch_release_frozen)
)

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
start_epoch = 0
epochs = 200

model.fit(
    train_data_loader,
    validation_data=valid_data_loader,
    epochs=epochs,
    callbacks=[reduceLROnPlat, save_c, csv_logger, make_trainable_callback],
    initial_epoch=start_epoch
)

Epoch 1/200
269/269 [==============================] - 388s 1s/step - loss: 0.7165 - accuracy: 0.5781 - val_loss: 4.4976 - val_accuracy: 0.5656
Epoch 2/200
269/269 [==============================] - 338s 1s/step - loss: 0.6632 - accuracy: 0.6010 - val_loss: 1.8267 - val_accuracy: 0.5969
Epoch 3/200
269/269 [==============================] - 362s 1s/step - loss: 0.6558 - accuracy: 0.5993 - val_loss: 1.8606 - val_accuracy: 0.5744
Epoch 4/200
269/269 [==============================] - 314s 1s/step - loss: 0.6391 - accuracy: 0.6141 - val_loss: 0.6142 - val_accuracy: 0.5956
Epoch 5/200
269/269 [==============================] - 313s 1s/step - loss: 0.6311 - accuracy: 0.6220 - val_loss: 0.6634 - val_accuracy: 0.5706
Epoch 6/200
269/269 [==============================] - 316s 1s/step - loss: 0.6290 - accuracy: 0.6346 - val_loss: 0.5837 - val_accuracy: 0.6844
Epoch 7/200
269/269 [==============================] - 313s 1s/step - loss: 0.6239 - accuracy: 0.6437 - val_loss: 1.8186 - val_accuracy:

269/269 [==============================] - 315s 1s/step - loss: 0.3832 - accuracy: 0.7974 - val_loss: 0.5738 - val_accuracy: 0.7981
Epoch 58/200
269/269 [==============================] - 315s 1s/step - loss: 0.3902 - accuracy: 0.7911 - val_loss: 0.7208 - val_accuracy: 0.8144
Epoch 59/200
269/269 [==============================] - 314s 1s/step - loss: 0.3982 - accuracy: 0.7933 - val_loss: 0.6223 - val_accuracy: 0.8106
Epoch 60/200
269/269 [==============================] - 314s 1s/step - loss: 0.3789 - accuracy: 0.8037 - val_loss: 0.4301 - val_accuracy: 0.8569
Epoch 61/200
269/269 [==============================] - 314s 1s/step - loss: 0.3804 - accuracy: 0.8001 - val_loss: 0.6662 - val_accuracy: 0.8481
Epoch 62/200
269/269 [==============================] - 314s 1s/step - loss: 0.3741 - accuracy: 0.8068 - val_loss: 0.4624 - val_accuracy: 0.8500
Epoch 63/200
269/269 [==============================] - 314s 1s/step - loss: 0.3678 - accuracy: 0.8060 - val_loss: 0.2729 - val_accuracy: 0.912

In [7]:
np.unique(list(valid_data_loader.data_getter.class_dict.values()))

array([0, 1])